# Coursework Part 1 (Solution): Detecting Spam with Spark

These are model answers for the IN432 Big Data coursework 2018, part 1.  

This coursework is about classification of e-mail messages as spam or non-spam in Spark. We will go through the whole process from loading preprocessing to training and testing classifiers in a distributed way in Spark. We wil use the techniques shown in the lextures and labs. I will also introduce here a few additional elements, such as the NLTK and some of the preprocessing and machine learning functions that come with Spark. You are not expected to need anything beyond the material handed out so far and in some cases the Spark documentation, to which I have put links in this document.  

The structure is similar to the lab sheets. I provide a code structure with gaps that you are supposed to file. In addition you should run 2 small experiments and comment on the results. The lines where you are supposed to add code or take another action are marked with ">>>" 
please leave the ">>>" in the text, comment out that line, and write your own code in the next line using a copy of that line as a starting point.

I have added numerous comments in text cells and the code cells to guid you through the program. Please read them carefully and ask if anything is unclear. 

Once you have completed the tasks, don't delete the outpus, but downlaod the notebook (outputs will be included) and upload it into the coursework submission area on Moodle. The coursework part counts for 25% or the total coursework.

## Load and prepare the data

We will use the lingspam dataset in this coursework (see [http://csmining.org/index.php/ling-spam-datasets.html](http://csmining.org/index.php/ling-spam-datasets.html) for more information).

The next cell is only needed if you haven't cloned the repository in week 2 or later (but it doesn't do harm to run it). 

In [ ]:
# go to the work directory 
%cd ~/notebook/work/
# and clone the project from github
!git clone https://github.com/tweyde/City-Data-Science.git

In [ ]:
# now that we have the project directory
%cd ~/notebook/work/City-Data-Science/
# we can pull the latest files
!git pull
#we need to use a magic command (starting with '%') here, to change the directory
%cd ./datasets/ 
print(">>> Extracting the ling_spam dataset, this can take a moment.")
# '!' calls a program on the machine (the DSX service runs on Linux machines). 
!tar -xf lingspam_public02.tar.gz
print(">>> Unzipping finished.")
# We now have a new dataset in directory 'bare'.
%cd lingspam_public/bare 
print(">>> pwd ")
!pwd
print(">>> ls ")
# the line before last of output should show "part1 part10 part2  part3  part4  part5  part6  part7 part8 part9"
!ls
# finally we return to the 'lingspam_public' directory
%cd ..

### Tools for Troubleshooting

Normally, DSX works reliably, but there are two issues that have occured. We have solutions for them that you can use with the following cells. 
If other problems occur, reloading the page and/or restarting the Kernel can help. 

In [25]:
# try this in case of "sc undefined" or similar errors, should normally not be necessary.
from pyspark import SparkContext 
sc = spark.sparkContext 

In [ ]:
# Sometimes you may get an error that the metastore_db is not accessible, especially when you have multiple notebooks open at the same time.
# We can not prevent this form happening on DSX.
# If it does happen you need to delete the metastore_db. 
# The path of the metastore_db is in the error messages and it's typically long like this example: 
# /gpfs/global_fs01/sym_shared/YPProdSpark/user/s832-dfe96c6e1f1d61-70d619a53771/notebook/jupyter-rt/kernel-cdcf5f73-9afb-481d-ac40-a210a649eb69-20180222_154448/metastore_db
# once you have it, you can use it with !rm -Rf to delete it (replace the term in <> with the path you get from your error message):
!rm -Rf <Put the path of the metastore_db here>

# Task 1) Read the dataset and create RDDs 
a) Start by reading the directory with text files from the file system (`~/notebook/work/City-Data-Science/datasets/bare`). Load all text files per dirctory (part1,part2, ... ,part10) using `wholeTextFiles()`, which creates one RDD per part, containing tuples (filename,text). This is a good choice as the text files are small. (5%)

b) We will use one of the RDDs as test set, the rest as training set. For the training set you need to create the union of the remaining RDDs. (5%)

b) Remove the path and extension from the filename using the regular expression provided (5%).

If the filename starts with 'spmsg' it is spam, otherwise it is not. We'll use that later to train a classifier. 

We will put the code in each cell into a function that we can reuse later. In this way we can develop the whole preprocessing with the smaller test set and apply it to the training set once we know that everything works. 

In [1]:
from pathlib import Path
import os.path
import re

def makeTestTrainRDDs(pathString):
    """ Takes one of the four subdirectories of the lingspam dataset and returns two RDDs one each for testing and training. """
    # We should see10 parts that we can use for creating train and test sets.
    p = Path(pathString) # gets a path object representing the current directory path.
    dirs = list(p.iterdir()) # get the directories part1 ... part10. 
    rddList = [] # create a list for the RDDs
    # now create an RDD for each 'part' directory and add them to rddList
    for d in dirs: # iterate through the directories
        rdd = sc.wholeTextFiles(str(d.absolute())) #>>> # read the files in the directory 
        rddList.append(rdd) #>>> append the RDD to the list
    #print('len(rddList)',len(rddList))  # we should now have 10 RDDs in the list # just for testing
    #print(rddList[1].take(1)) # just for testing, comment out when it works.
    testRDD1 = rddList[9] # set the test set
    trainRDD1 = rddList[0] # start the training set from 0 and 
    # now loop over the range from 1 to 9(exclusive) to create a union of the remaining RDDs
    for i in range(1,9):
        trainRDD1 = trainRDD1.union(rddList[i]) #>>> create a union of the current and the next 
            # RDD in the list, so that in the end we have a union of all parts 0-8. (9 ist used as test set)
    # both RDDs should remove the paths and extensions from the filename. 
    # This regular expression will do it: re.split('[/\.]', fn_txt[0])[-2]
    #>>> apply it to the filenames in train and test RDD with a lambda
    testRDD2 = testRDD1.map(lambda fn_txt: (re.split('[/\.]', fn_txt[0])[-2], fn_txt[1])) # >>>
    trainRDD2 = trainRDD1.map(lambda fn_txt: (re.split('[/\.]', fn_txt[0])[-2], fn_txt[1])) # >>>
    return (trainRDD2,testRDD2)

# dictionary for caching RDDs for multiple use
tmp1 = {} 

# This is a different version, which creates the training RDD in one operation using glob expressions instead of a union of RDDs and caching
def makeTestTrainRDDs2(pathString):
    """ Takes one of the four subdirectories of the lingspam dataset and returns two RDDs one each for testing and training. 
    It caches the data RDDs, so that within one process they do not need   """ 
    if pathString not in tmp1:
        # We should see10 parts that we can use for creating train and test sets
        absPath = os.path.abspath(pathString)
        testRDD1 = sc.wholeTextFiles(absPath+'/part10/')
        trainRDD1 = sc.wholeTextFiles(absPath+'/part[1-9]/')
        # both RDDs should remove the paths and extensions from the filename. 
        # This regular expression will do it: re.split('[/\.]', fn_txt[0])[-2]
        #>>> apply it to the filenames in train and test RDD with a lambda
        testRDD2 = testRDD1.map(lambda fn_txt: (re.split('[/\.]', fn_txt[0])[-2], fn_txt[1])) # >>>
        trainRDD2 = trainRDD1.map(lambda fn_txt: (re.split('[/\.]', fn_txt[0])[-2], fn_txt[1])) # >>>
        testRDD2.cache()
        testRDD2.cache()
        tmp1[pathString] = (trainRDD2,testRDD2)
    return tmp1[pathString]


# this makes sure we are in the right directory
%cd ~/notebook/work/City-Data-Science/datasets/lingspam_public/
# this should show "bare  lemm  lemm_stop  readme.txt  stop"
!ls
# the code below is for testing the function makeTestTrainRDDs
trainRDD_testRDD = makeTestTrainRDDs('bare') # read from the 'bare' directory - this takes a bit of time
(trainRDD,testRDD) = trainRDD_testRDD # unpack the returned tuple
print('created the RDDs') # notify the user, so that we can figure out where things went wrong if they do.
print('testRDD.count(): ',testRDD.count()) # should be ~290 
#print('trainRDD.count(): ',trainRDD.count()) # should be ~2600 - commented out to save time
print('testRDD.getNumPartitions()',testRDD.getNumPartitions()) # normally 2 on DSX
print('testRDD.getStorageLevel()',testRDD.getStorageLevel()) # Serialized 1x Replicated on DSX
print('testRDD.take(1): ',testRDD.take(1)) # should be (filename,[tokens]) 

rdd1 = testRDD # use this for developemnt in the next tasks 

/gpfs/global_fs01/sym_shared/YPProdSpark/user/s832-dfe96c6e1f1d61-70d619a53771/notebook/work/City-Data-Science/datasets/lingspam_public
bare  lemm  lemm_stop  readme.txt  results1.pkl  results2.pkl  stop
created the RDDs
testRDD.count():  291
testRDD.getNumPartitions() 2
testRDD.getStorageLevel() Serialized 1x Replicated
testRDD.take(1):  [('9-66msg1', "Subject: xth conference of nordic and general ling .\n\nthe tenth conference of nordic and general linguistics will be held in reykjavik , iceland , from saturday june 6 , to monday june 8 , 1998 . it is organized by the institute of linguistics , university of iceland . the deadline for pre-registration at a reduced price is january 31 , 1998 . pre - registration forms and further information can be found on our web site ( http : / / www . rhi . hi . is / ~ nordconf ) and can also be mailed or e-mailed upon request . papers on any linguistic topic are invited , especially papers on synchronic and diachronic aspects of the nordic langua

### Task 2) Tokenize and remove punctuation

Now we need to split the words, a process called *tokenization* by linguists, and remove punctuation. 

We will use the Python [Natural Language Toolkit](http://www.nltk.org) *NLTK* to do the tokenization (rather than splitting ourselves, as these specialist tools usually do that better than we can ourselves). We use the NLTK function word_tokenize, see here for a code example: [http://www.nltk.org/book/ch03.html](http://www.nltk.org/book/ch03.html). (5%)

Then we will remove punctuation. There is no specific funtion for this, so we use a regular expression (see here for info [https://docs.python.org/3/library/re.html?highlight=re#module-re](https://docs.python.org/3/library/re.html?highlight=re#module-re)) in a list comprehension (here's a nice visual explanation: [http://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/](http://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/)). (5%) 

We use a new technique here: we separate keys and values of the RDD, using the RDD functions `keys()` and `values()`, which yield each a new RDD. Then we process the values and *zip* them together with the keys again. See here for documentation: [http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.zip](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.zip).  We wrap the whole sequence into one function `prepareTokenRDD` for later use. (5%)

In [2]:
import nltk    
import re
from nltk.corpus import stopwords

def tokenize(text):
    """ Apply the nltk.word_tokenize() method to our text, return the token list. """
    nltk.download('punkt') # this loads the standard NLTK tokenizer model 
    # it is important that this is done here in the function, as it needs to be done on every worker.
    # If we do the download outside a this function, it would only be executed on the driver     
    return nltk.word_tokenize(text) # >>> use the nltk function word_tokenize
    
def removePunctuation(tokens):
    """ Remove punctuation characters from all tokens in a provided list. """
    tokens2 =  [re.sub('[()\[\],.?!";_]','',token) for token in tokens] # use a list comprehension to remove punctuaton
    return tokens2

def prepareTokenRDD(fn_txt_RDD):
    """ Take an RDD with (filename,text) elements and transform it into a (filename,[token ...]) RDD without punctuation characters. """
    rdd_vals2 = fn_txt_RDD.values() # It's convenient to process only the values. 
    rdd_vals3 = rdd_vals2.map(tokenize) # Create a tokenised version of the values by mapping (we could have done that in palce with a lambda)
    rdd_vals4 = rdd_vals3.map(removePunctuation) # remove punctuation from the values (could also have been a lambda)
    rdd4 = fn_txt_RDD.keys().zip(rdd_vals4) # we zip the two RDDs together 
    # i.e. produce tuples with one item from each RDD.
    # This works because we have only applied mappings to the values, 
    # therefore the items in both RDDs are still aligned.
    # >>> now remove any empty strings (i.e. length 0) that we may have 
    # created by removing punctuation, and resulting entries without words left.
    rdd5 = rdd4.map(lambda kv: (kv[0],[s for s in kv[1] if len(s)>0])) # remove empty words using RDD.map and a lambda. 
    rdd6 = rdd5.filter(lambda kv: len(kv[0])>0) # remove empty items using RDD.filter and a lambda. 
    # >>> Question: why should this filtering be done after zipping the keys and values together?
    return rdd6 

rdd2 = prepareTokenRDD(rdd1) # Use the test set for now, because it is smaller
print(rdd2.take(1)) #  checking result of task 2. 
# The output should looks something like: "[('9-66msg1', ['Subject', ':', 'xth', 'conference', 'of', 'nordic', 'and', 'general', 'ling', ... "

[('9-66msg1', ['Subject', ':', 'xth', 'conference', 'of', 'nordic', 'and', 'general', 'ling', 'the', 'tenth', 'conference', 'of', 'nordic', 'and', 'general', 'linguistics', 'will', 'be', 'held', 'in', 'reykjavik', 'iceland', 'from', 'saturday', 'june', '6', 'to', 'monday', 'june', '8', '1998', 'it', 'is', 'organized', 'by', 'the', 'institute', 'of', 'linguistics', 'university', 'of', 'iceland', 'the', 'deadline', 'for', 'pre-registration', 'at', 'a', 'reduced', 'price', 'is', 'january', '31', '1998', 'pre', '-', 'registration', 'forms', 'and', 'further', 'information', 'can', 'be', 'found', 'on', 'our', 'web', 'site', 'http', ':', '/', '/', 'www', 'rhi', 'hi', 'is', '/', '~', 'nordconf', 'and', 'can', 'also', 'be', 'mailed', 'or', 'e-mailed', 'upon', 'request', 'papers', 'on', 'any', 'linguistic', 'topic', 'are', 'invited', 'especially', 'papers', 'on', 'synchronic', 'and', 'diachronic', 'aspects', 'of', 'the', 'nordic', 'languages', 'invited', 'speakers', ':', 'anders', 'holmberg', 't

### Task 3) Creating normalised TF.IDF vectors of defined dimensionality, measure the effect of caching.

We use the hashing trick to create fixed size TF vectors directly from the word list now (slightly different from the previous lab, where we used *(word,count)* pairs.). Write a bit of code as needed. (5%)

Then we'll use the IDF and Normalizer functions provided by Spark. They use a slightly different pattern than RDD.map and reduce, have a look at the examples here in the documentation for Normalizer  and IDF:
[http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.Normalizer](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.Normalizer), [http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.IDF](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.IDF) (5%)

We want control of the dimensionality in the `normTFIDF` function, so we introduce an argument into our functions that enables us to vary dimensionalty later. Here is also an opportunity to benefit from caching, i.e. persisting the RDD after use, so that it will not be recomputed.  (5%)

In [3]:
# use the hashing trick to create a fixed-size vector from a word list
def hashing_vectorize(text,N): # arguments: the list and the size of the output vector
    v = [0] * N  # create vector of 0s
    for word in text: # iterate through the words 
        h = hash(word)          # get the hash value 
        v[h % N] = v[h % N] + 1 # add 1 at the hashed address 
    return v # return hashed word vector

from pyspark.mllib.feature import IDF, Normalizer

tmp2 = {}
def normTFIDF(fn_tokens_RDD, vecDim, caching=True):
    keysRDD = fn_tokens_RDD.keys()
    tokensRDD = fn_tokens_RDD.values()
    tfVecRDD = tokensRDD.map(lambda tokens: hashing_vectorize(tokens,vecDim)) #>>> passing the vecDim value. TIP: you need a lambda. 
    if caching:
        tfVecRDD.persist(StorageLevel.MEMORY_ONLY) # since we will read more than once, caching in Memory will make things quicker.
    idf = IDF()
    idfModel = idf.fit(tfVecRDD)
    tfIdfRDD = idfModel.transform(tfVecRDD)
    norm = Normalizer()
    normTfIdfRDD = norm.transform(tfIdfRDD)
    zippedRDD = keysRDD.zip(normTfIdfRDD)
    return zippedRDD

testDim = 10 # too small, but good for testing
rdd3 = normTFIDF(rdd2, testDim, True) # apply task2
print(rdd3.take(10)) # we should now have tuples with ('filename',[N-dim vector])
# e.g. [('9-1142msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.4097, 0.0, 0.0, 0.0, 0.9122, 0.0]))]

[('9-66msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-751msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('spmsgc6', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-628msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-976msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-973msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-839msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-709msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-922msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])), ('9-842msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]))]


## Comment:
The zeros here appear because the log(IDF) values will be 0 for many dimensions. This occurs because we calculate the Hashing Vector before we calculate the IDF valuest. This is more efficient than calculating TF.IDF first, but leads to quicker degradation when the vector dimensions are small. This makes finding an appropriate vector size even more important. 

#### Task 3a) Caching experiment

The normTFIDF let's us switch caching on or off. Write a bit of code that measures the effect of caching by takes the time for both options. Use the time function as shown in lecture 3, slide 47. Remember that you need to call an action on an RDD to trigger full execution. 

Add a short comment on the result (why is there an effect, why of the size that it is?). Remember that this is wall clock time, i.e. you may get noisy results. (10%)

In [ ]:
#run a small experiment with caching set to True or False, 3 times each

from time import time

resCaching = [] # for storing results
resNoCache = []
for i in range(3): # 3 samples
    startTime = time() # start timer
    testRDD1 = normTFIDF(rdd2, testDim, True) # 
    testRDD1.collect() # needed to force execution
    endTime = time()
    resCaching.append( endTime - startTime )
    
    startTime = time()
    testRDD2 = normTFIDF(rdd2, testDim, False) 
    testRDD2.collect() # needed to force execution   
    endTime = time()
    resNoCache.append( endTime - startTime )
    
meanTimeCaching = sum(resCaching)/len(resCaching)
meanTimeNoCache = sum(resNoCache)/len(resNoCache)

print('Creating TF.IDF vectors, 3 trials - mean time with caching: ', meanTimeCaching, ', mean time without caching: ', meanTimeNoCache)

## Comment
My results are:
`Creating TF.IDF vectors, 3 trials - mean time with caching:  15.675768772761026 , mean time without caching:  22.62595049540202`

This shows that caching speeds up the processing. This is because the calculations and reading from disk would have to 
be preformed again for 2nd use if there is no cache. Persistence means that the 2nd time round we can read stored valued from memory. 
This is very effective here, as the RDD is small and will fit into memory. 
There are two times that tfVecRDD is read when the IDF is calculated and applied, by persisting (caching) we calculate it only once, which reduces the time taken by about 30%.

## Task 4) Create LabeledPoints 

Determine whether the file is spam (i.e. the filename contains ’spmsg’) and replace the filename by a 1 (spam) or 0 (non-spam) accordingly. Use `RDD.map()` to create an RDD of LabeledPoint objects. See here [http://spark.apache.org/docs/2.1.0/mllib-linear-methods.html#logistic-regression](http://spark.apache.org/docs/2.1.0/mllib-linear-methods.html#logistic-regression) for an example, and here [http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LabeledPoint](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LabeledPoint) for the `LabeledPoint` documentation. (10%)

There is a handy function of Python strings called startswith: e.g. 'abc'.startswith('ab) will return true. The relevant Python syntax here is a conditional expression: **``<a> if <yourCondition> else <b>``**, i.e. 1 if the filename starts with 'spmsg' and otherwise 0.

In [31]:
from pyspark.mllib.regression import LabeledPoint

# creatate labelled points of vector size N out of an RDD with normalised (filename [(word,count), ...]) items
def makeLabeledPoints(fn_vec_RDD): # RDD and N needed 
    # we determine the true class as encoded in the filename and represent as 1 (samp) or 0 (good)
    cls_vec_RDD = fn_vec_RDD.map(lambda fn_vec: (1 if fn_vec[0].startswith('spmsg') else 0,fn_vec[1])) 
    # now we can create the LabeledPoint objects with (class,vector) arguments
    lp_RDD = cls_vec_RDD.map(lambda cls_vec: LabeledPoint(cls_vec[0],cls_vec[1]) ) 
    return lp_RDD 

# for testing
testLpRDD = makeLabeledPoints(rdd3) 
print(testLpRDD.take(1)) 
# should look like this: [LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.40968062880166006,0.0,0.0,0.0,0.9122290186048,0.0])]

[LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012736603569842138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022145734121861216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015008921926629391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013356428624859975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014571310671296732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031132612242499597,0.0,0.0,0.0,0.0,0.033358503557829565,0.0,0.0,0

### Complete the preprocessing (no task, but read it)

It will be useful to have a single function to do the preprocessing. Se integrate everything here.

In [32]:
# now we can apply the preprocessing chain to the data loaded in task 1 
# N is for controlling the vector size
def preprocess(rawRDD,N):
    """ take a (filename,text) RDD and transform into LabelledPoint objects 
        with class labels and a TF.IDF vector with N dimensions. 
    """
    tokenRDD = prepareTokenRDD(rawRDD) # task 2
    tfIdfRDD = normTFIDF(tokenRDD,N) # task 3
    lpRDD = makeLabeledPoints(tfIdfRDD) # task 4
    return lpRDD # return RDD with LabeledPoints

# and with this we can start the whole process from a directory, N is again the vector size
def loadAndPreprocess(directory,N):
    """ load lingspam data from a directory and create a training and test set of preprocessed data """
    trainRDD_testRDD = makeTestTrainRDDs(directory) # read from the directory 
    (trainRDD,testRDD) = trainRDD_testRDD # unpack the returned tuple
    return (preprocess(trainRDD,N),preprocess(testRDD,N))

trainLpRDD = preprocess(trainRDD,testDim) # prepare the training data
print(testLpRDD.take(1)) # should look similar to the previous cell

train_test_100_LpRDD = loadAndPreprocess('lemm',100) # let's re-run with another vector size
(trainLpRDD,testLpRDD) = train_test_100_LpRDD
print(testLpRDD.take(1))
print(trainLpRDD.take(1))

[LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012736603569842138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022145734121861216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015008921926629391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013356428624859975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014571310671296732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031132612242499597,0.0,0.0,0.0,0.0,0.033358503557829565,0.0,0.0,0

[LabeledPoint(0.0, [0.01730655982377985,0.045028204382840185,0.07637472435997773,0.042890046052874164,0.07302782181622049,0.1384524785902388,0.2074681106135228,0.09074474397362073,0.09984843251900194,0.1597574920304031,0.056829972009760706,0.17574880095731008,0.1736507408364363,0.0,0.08419904896411998,0.03185335921294318,0.02714910064882251,0.02239724489096134,0.02886131863018766,0.06502315505843996,0.2375234915140254,0.04312192975940643,0.0,0.13404945757161074,0.0,0.083152320643562,0.09855693351863973,0.023617160959519868,0.06588533144332052,0.14186982096498008,0.13858720107260336,0.07675484158875623,0.08172924859742939,0.12940815304264155,0.1609541271472273,0.0,0.12005972919333587,0.11659715351764918,0.04209952448205999,0.0692262392951194,0.054655819097232855,0.0692262392951194,0.029389498005929306,0.08807842815688952,0.1486029062380973,0.14718449105986933,0.045372371986810364,0.03647997801367858,0.1437401368593856,0.03353587799378589,0.17972717853420347,0.10419044450186177,0.0732586

## Task 6) Train some classifiers 

Use the `LabeledPoint` objects to train a classifier, specifically the *LogisticRegression*, *Naive Bayes*, and *Support Vector Machine*. Calculate the accuracy of the model on the training set (again, follow this example [http://spark.apache.org/docs/2.1.0/ml-classification-regression.html#logistic-regression](http://spark.apache.org/docs/2.0.0/ml-classification-regression.html#logistic-regression) and here is the documentation for the classifiers [LogisticRegressionWithLBFGS](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithLBFGS), [NaiveBayes](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.NaiveBayes), [SVMWithSGD](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.SVMWithSGD).  (10%) 

In [36]:
from pyspark.mllib.classification import (NaiveBayes, LogisticRegressionWithLBFGS, SVMWithSGD) 
import numpy

# train the model with an (f,[(w,c), ...]) RDD. This is practical as we can reuse the function for TF.IDF
def trainModel(lpRDD):
    """ Train 3 classifier models on the given RDD with LabeledPoint objects. A list of trained model is returned. """
    lpRDD.persist(StorageLevel.MEMORY_ONLY) # not really needed as the Spark implementations ensure caching themselves. 
                    # Other implementations might not, however. 
    # Train a classifier model.
    print('Starting to train the models') # give some immediate feedback
    model1 = LogisticRegressionWithLBFGS.train(lpRDD) # this is the best model
    print('Trained LR (model1)')
    #print('type(model1)')
    model2 = NaiveBayes.train(lpRDD) # doesn't work well. 
    print('Trained NB (model2)')
    #print(type(model2))
    model3 = SVMWithSGD.train(lpRDD) # or this ... doesn't do well either.
    print('Trained SVM (model3)')
    return [model1,model2,model3]

def testModel(model, lpRDD):
    """ Tests the classificatio accuracy of the given model on the given RDD with LabeledPoint objects. """
    lpRDD.persist(StorageLevel.MEMORY_ONLY)
    # Make prediction and evaluate training set accuracy.
    # Get the prediction and the ground truth label
    predictionAndLabel = lpRDD.map(lambda p: (model.predict(p.features), p.label)) # get the prediction and ground truth (label) for each item.
    correct = predictionAndLabel.filter(lambda xv: xv[0] == xv[1]).count() # count the correct predictions 
    accuracy = 1.0 * correct / lpRDD.count() # and calculate the accuracy 
    print('Accuracy {:.2%} (data items: {}, correct: {})'.format(accuracy,lpRDD.count(), correct)) # report to console
    return accuracy # and return the value  

models = trainModel(trainLpRDD) # just for testing
testModel(models[2], trainLpRDD) # just for testing

Starting to train the models
Trained LR (model1)
Trained NB (model2)
Trained SVM (model3)
Accuracy 83.40% (data items: 2602, correct: 2170)


0.8339738662567256

### Task 7) Automate training and testing

We automate now the whole process from reading the files, through preprocessing, and training up to evaluating the models. In the end we have a single function that takes all the parameters we are interested in and produces trained models and an evaluation. (5%) 

In [37]:
# this function combines tasks f) and g)
# this method should take RDDs with (f,[(w,c), ...])
def trainTestModel(trainRDD,testRDD):
    """ Trains 3 models and tests them on training and test data. Returns a matrix the training and testing (rows) accuracy values for all models (columns). """
    models = trainModel(trainRDD)
    results = [[],[]] # matrix for 2 modes (training/test) vs n models (currently 3)
    for mdl in models:
        print('Training')
        results[0].append(testModel(mdl, trainRDD))
        print('Testing')
        results[1].append(testModel(mdl, testRDD))
    return results

def trainTestFolder(folder,N):
    """ Reads data from a folder, preproceses the data, and trains and evaluates models on it. """
    print('Start loading and preprocessing') 
    train_test_LpRDD = loadAndPreprocess(folder,N) # create the RDDs
    print('Finished loading and preprocessing')
    (trainLpRDD,testLpRDD) = train_test_LpRDD # unpack the RDDs 
    return trainTestModel(trainLpRDD,testLpRDD) # train and test

trainTestFolder('lemm',1000) 

Start loading and preprocessing
Finished loading and preprocessing
Starting to train the models
Trained LR (model1)
Trained NB (model2)
Trained SVM (model3)
Training
Accuracy 100.00% (data items: 2602, correct: 2602)
Testing
Accuracy 97.59% (data items: 291, correct: 284)
Training
Accuracy 94.31% (data items: 2602, correct: 2454)
Testing
Accuracy 92.10% (data items: 291, correct: 268)
Training
Accuracy 83.40% (data items: 2602, correct: 2170)
Testing
Accuracy 83.16% (data items: 291, correct: 242)


[[1.0, 0.9431206764027671, 0.8339738662567256],
 [0.9759450171821306, 0.9209621993127147, 0.8316151202749141]]

## Task 8) Run experiments 

We have now a single function that allows us to vary the vector size easily. Test vector sizes 3, 30, 300, 3000, 30000 and examine the effect on the classification accuracy in Experiment 1.

Use the function from Task 7) to test different data types. The dataset has raw text in folder `bare`, lemmatised text in  `lemm` (similar to stemming, reduces to basic word forms), `stop` (with stopwords removed), and `lemm_stop` (lemmatised and stopwords removed). Test how the classification accuracy differs for these four data types in Experiment 2. 

Comment on the results in a few sentences, considering the differences in performance between the different conditions as well as train and test values. 15%

In [39]:
folder = 'bare'
N = numpy.array([3,30,300,3000,30000]) 
print('\nEXPERIMENT 1: Testing different vector sizes')
results1 = []
for n in N:
    print('N = {}'.format(n))
    result = {'n':n, 't':folder}
    result['acc'] = trainTestFolder(folder,n)
    results1.append(result)
    
n = 3000
print('EXPERIMENT 2: Testing different data types')
results2 = []
typeFolders = ['bare','stop','lemm','lemm_stop']
for folder in typeFolders:
    print('Path = {}'.format(folder))
    result = {'n':n, 't':folder}
    result['acc'] = trainTestFolder(folder,n)
    results2.append(result)

# This wasn't asked for, but can be useful if you want to analyse the results later: 
import pickle as pkl
with open('results1.pkl','wb') as f:
    pkl.dump(results1,f)
with open('results2.pkl','wb') as f:
    pkl.dump(results2,f)


EXPERIMENT 1: Testing different vector sizes
N = 3
Start loading and preprocessing
Finished loading and preprocessing
Starting to train the models
Trained LR (model1)
Trained NB (model2)
Trained SVM (model3)
Training
Accuracy 83.40% (data items: 2602, correct: 2170)
Testing
Accuracy 83.16% (data items: 291, correct: 242)
Training
Accuracy 83.40% (data items: 2602, correct: 2170)
Testing
Accuracy 83.16% (data items: 291, correct: 242)
Training
Accuracy 83.40% (data items: 2602, correct: 2170)
Testing
Accuracy 83.16% (data items: 291, correct: 242)
N = 30
Start loading and preprocessing
Finished loading and preprocessing
Starting to train the models
Trained LR (model1)
Trained NB (model2)
Trained SVM (model3)
Training
Accuracy 85.47% (data items: 2602, correct: 2224)
Testing
Accuracy 86.60% (data items: 291, correct: 252)
Training
Accuracy 83.40% (data items: 2602, correct: 2170)
Testing
Accuracy 83.16% (data items: 291, correct: 242)
Training
Accuracy 83.40% (data items: 2602, correct:

## Comment
The test and training accuracy increase with the vector size. This is because we loose information in the hashing process, and in addition by the IDF calculation on small vectors. For vecture sizes of 3000 and higher, Logistic Regression produces perfect classification on the Training set, and reaches over 98% for 30000 vector dimensions. 

For the different preprocessing types (bare, stop, lemm, lemm_stop) it is not clear a priori, which of them might be more effective, as spam can be expected to be different in terms of topics and style, which meanse that may contain relevant information. In practice the preprocessing has less impact on model performance than the vector size, the accuracy is similar in all 4 conditions. Specifically for the best model, Logistic Regression, the test accuracy is 97.59% in all cases but lemm_stop, where it is 97.94. 

In [8]:
# This wasn't asked for but it is a nice way to show the results graphically.
from pyspark.sql import DataFrame
import pandas as pd
import pixiedust

import pickle as pkl
with open('results1.pkl','rb') as f:
    results1 = pkl.load(f)

with open('results2.pkl','rb') as f:
    results2 = pkl.load(f)


#We'll create a structured data frame of our results, for that we'll provide all 
models = ['LR','NB','SVM']
modes = ['Train','Test']

def makePandasDf():
    df = pd.DataFrame(columns=['t','model','n','mode','acc'])
    for r in results1:
        for tst in (0,1): 
            for mod in (0,1,2):
                r2 = r.copy() 
                r2['model']=models[mod]
                r2['mode']=modes[tst]
                r2['acc']=r['acc'][tst][mod]
                #print(r2)
                df = df.append(r2, ignore_index=True)

    df2 = pd.DataFrame(columns=['t','model','n','mode','acc'])
    for r in results2:
        for tst in (0,1): 
            for mod in (0,1,2):
                r2 = r.copy() 
                r2['model']=models[mod]
                r2['mode']=modes[tst]
                r2['acc']=r['acc'][tst][mod]
                #print(r2)
                df2 = df.append(r2, ignore_index=True)

    return df,df2
        
#sdf = makePandasDf(res3a)
display(makePandasDf()[0])